In [3]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.0.0
1.17.2


## Eager execution

In [4]:
a = tf.constant([[10,10],[11.,1.]])
x = tf.constant([[1.,0.],[0.,1.]])
b = tf.Variable(12.)
y = tf.matmul(a, x) + b
print(y.numpy())

[[22. 22.]
 [23. 13.]]


## tf.function, not tf.Session

In [17]:
def f():
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.)
    y = tf.matmul(a, x) + b
    return y

In [18]:
print(f().numpy())

[[22. 22.]
 [23. 13.]]


In [37]:
@tf.function
def f():
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.)
    y = tf.matmul(a, x) + b
    print(f'PRINT: {y}')
    tf.print(f'TF-PRINT: {y}')
    return y
f()

PRINT: Tensor("add:0", shape=(2, 2), dtype=float32)


ValueError: in converted code:

    <ipython-input-21-64a4c693dc21>:5 f  *
        b = tf.Variable(12.)
    /home/matthew/test/2tense2flow/env/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /home/matthew/test/2tense2flow/env/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /home/matthew/test/2tense2flow/env/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /home/matthew/test/2tense2flow/env/lib/python3.6/site-packages/tensorflow_core/python/eager/def_function.py:413 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


Because `f` contains `tf.Variable` definition

## Solution

Object Oriented

In [52]:
class F():
    def __init__(self):
        self._b = None
        
    @tf.function
    def __call__(self):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])
        if self._b is None:
            self._b = tf.Variable(12.)
        y = tf.matmul(a, x) + self._b
        print(f'PRINT: {y}')
        tf.print(f'TF-PRINT: {y}')
        return y
    
f = F()
f()

PRINT: Tensor("add:0", shape=(2, 2), dtype=float32)
PRINT: Tensor("add:0", shape=(2, 2), dtype=float32)
TF-PRINT: Tensor("add:0", shape=(2, 2), dtype=float32)


<tf.Tensor: id=612, shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

Var as input

In [50]:
@tf.function
def f(b):
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    y = tf.matmul(a, x) + b
    print("PRINT: ", y)
    tf.print("TF-PRINT: ", y)
    return y

b = tf.Variable(12.)
f(b)

PRINT:  Tensor("add:0", shape=(2, 2), dtype=float32)
TF-PRINT:  [[22 22]
 [23 13]]


<tf.Tensor: id=562, shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

## Input type

In [76]:
@tf.function
def f(x):
    print("Python execution: ", x)
    tf.print("Graph execution: ", x)
    return x

In [72]:
print("##### float32 test #####")
a = tf.constant(1, dtype=tf.float32)
print("first call")
f(a)
a = tf.constant(1.1, dtype=tf.float32)
print("second call")
f(a)

print("##### uint8 test #####")

b = tf.constant(2, dtype=tf.uint8)
print("first call")
f(b)
b = tf.constant(3, dtype=tf.uint8)
print("second call")
f(b)

##### float32 test #####
first call
Python execution:  Tensor("x:0", shape=(), dtype=float32)
Graph execution:  1
second call
Graph execution:  1.1
##### uint8 test #####
first call
Python execution:  Tensor("x:0", shape=(), dtype=uint8)
Graph execution:  2
second call
Graph execution:  3


<tf.Tensor: id=775, shape=(), dtype=uint8, numpy=3>

In [73]:
# graph version of f
tf.autograph.to_code(f.python_function)

"def tf__f(x):\n  do_return = False\n  retval_ = ag__.UndefinedReturnValue()\n  with ag__.FunctionScope('f', 'f_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as f_scope:\n    print('Python execution: ', x)\n    ag__.converted_call(tf.print, f_scope.callopts, ('Graph execution: ', x), None, f_scope)\n    do_return = True\n    retval_ = f_scope.mark_return_value(x)\n  do_return,\n  return ag__.retval(retval_)\n"

## Python native type

In [77]:
def printinfo(x):
    print("Type: ", type(x), " value: ", x)

print("##### int test #####")
print("first call")
a = 1
printinfo(a)
f(a)
print("second call")
b = 2
printinfo(b)
f(b)

print("##### float test #####")
print("first call")
a = 1.0
printinfo(a)
f(a)
print("second call")
b = 2.0
printinfo(b)
f(b)

print("##### complex test #####")
print("first call")
a = complex(1.0, 2.0)
printinfo(a)
f(a)
print("second call")
b = complex(2.0, 1.0)
printinfo(b)
f(b)

##### int test #####
first call
Type:  <class 'int'>  value:  1
Python execution:  1
Graph execution:  1
second call
Type:  <class 'int'>  value:  2
Python execution:  2
Graph execution:  2
##### float test #####
first call
Type:  <class 'float'>  value:  1.0
Graph execution:  1
second call
Type:  <class 'float'>  value:  2.0
Graph execution:  2
##### complex test #####
first call
Type:  <class 'complex'>  value:  (1+2j)
Python execution:  (1+2j)
Graph execution:  (1+2j)
second call
Type:  <class 'complex'>  value:  (2+1j)
Python execution:  (2+1j)
Graph execution:  (2+1j)


<tf.Tensor: id=834, shape=(), dtype=complex128, numpy=(2+1j)>

In [78]:
ret = f(1.0)
if tf.float32 == ret.dtype:
    print("f(1.0) returns float")
else:
    print("f(1.0) return ", ret)

Graph execution:  1
f(1.0) return  tf.Tensor(1, shape=(), dtype=int32)


## Performance

In [83]:
import time

In [85]:
@tf.function
def g(x):
    return x

start = time.time()
for i in tf.range(100):
    g(i)
end = time.time()

print("tf.Tensor time elapsed: ", (end-start))

start = time.time()
for i in range(100):
    g(i)
end = time.time()

print("Native type time elapsed: ", (end-start))

tf.Tensor time elapsed:  0.05961346626281738


Native type time elapsed:  0.327803373336792


Use `tf.Tensor` everywhere

In [ ]:
d